# Large Context with Legal Documents Demo

## Install 

```
pip install jupyter pypdf validators
```

Optionally you can install tika, which requires Java installed. Tika reads pdf files as alternative to pypdf.
```
pip install tika
```

## Init

In [49]:
import os
import uuid
from pprint import pprint
from ai21 import AI21Client
from ai21.models.chat import ChatMessage, DocumentSchema

ai21_api_key = (os.environ.get("AI21_API_KEY"))

if not ai21_api_key:
    raise ValueError("Please set the environment variable AI21_API_KEY")

client = AI21Client(
    # This is the default and can be omitted
    api_key=ai21_api_key,
)

# list all files in a directory
def list_files(directory, extension=None):
    if extension:
        return [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f)) and f.endswith(extension)]
    else:
        return [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

# read pdf file and extract text
def read_pdf_file(file_name, preprocess=False, package="pypdf"):
    
    if package == "pypdf":
        return read_pdf_file_with_pypdf(file_name, preprocess)
    elif package == "tika":
        return read_pdf_file_with_tika(file_name, preprocess)
    else:
        raise ValueError(f"Package {package} not supported")
    

def read_pdf_file_with_pypdf(file_name, preprocess=False):
    from pypdf import PdfReader
    
    with open(file_name, 'rb') as f:
        reader = PdfReader(f)

        pypdf_content = ""
        for page in reader.pages:
            current_page = page.extract_text()
            pypdf_content += '\n\n' + current_page

        if preprocess:
            pypdf_content = re.sub(r'\s+', ' ', pypdf_content)

        return pypdf_content


def read_pdf_file_with_tika(file_name, preprocess=False):
    from tika import parser
    import re

    parsed_pdf = parser.from_file(file_name)
    tika_content = parsed_pdf['content']

    if preprocess:
        # remove extra spaces
        tika_content = re.sub(r'\s+', ' ', tika_content)

    return tika_content

## PDF Files

Large context comes from the 4 pdf documents - one per use case.

In [50]:
PDF_FILE_HOME = "../data/pdfs/"
PDF_FILES = [os.path.join(PDF_FILE_HOME, f) for f in list_files(PDF_FILE_HOME, "pdf")]

print(PDF_FILES)

pdf1_doc = read_pdf_file(PDF_FILES[3])
assert "DEPARTMENT OF THE TREASURY" in pdf1_doc[:100]
assert "Notice of proposed rulemaking and request for public comment." in pdf1_doc[:1000]
pdf1_doc = DocumentSchema(
    id=str(uuid.uuid4()),
    content=pdf1_doc,
)

pdf2_doc = read_pdf_file(PDF_FILES[2])
assert "KAR AUCTION SERVICES, INC." in pdf2_doc[:1000]
assert "CARVANA GROUP, LLC" in pdf2_doc[:1000]
pdf2_doc = DocumentSchema(
    id=str(uuid.uuid4()),
    content=pdf2_doc,
)

pdf3_doc = read_pdf_file(PDF_FILES[1])
assert "THOMAS HIGH PERFORMANCE GREEN FUND" in pdf3_doc[:10000]
pdf3_doc = DocumentSchema(
    id=str(uuid.uuid4()),
    content=pdf3_doc,
)

pdf4_doc = read_pdf_file(PDF_FILES[0])
assert "AAVANTIBIO" in pdf4_doc[:1000]
pdf4_doc = DocumentSchema(
    id=str(uuid.uuid4()),
    content=pdf4_doc,
)

['../data/pdfs/PDF Document4.pdf', '../data/pdfs/PDF Document3.pdf', '../data/pdfs/PDF Document2.pdf', '../data/pdfs/PDF Document.pdf']


## Prompt Setups

### PDF1 Use Case

In [51]:
initial_prompt1 = ChatMessage(
    role='user',
    content='''
    You are an M&A partner at a top global law firm. You should showcase expertise and capabilities when answering questions, and at the same time the answers should be digestible, similar in tone to conversation at a business lunch, and avoid using jargon.
    
    Draft a client alert (a three page write up that will be published online and sent to existing and potential clients to highlight my knowledge, insight, and expertise) about the proposed rule attached. The Proposed Rule revives a proposal first made in 2016 regarding rules on incentive-based compensation arrangements at certain financial institutions with at least $1 billion in assets, but newly includes a preamble that proposes certain alternatives and questions that will be considered for the final rule. The information in this preamble, which is under the heading of "Overview of the 2024 Proposed Rule" and the questions that will be considered for the final rule are very important to address in my client memo and should be discussed in the most detail. 

    Please draft this three page client alert, making sure to include the key provisions of the proposed rule, a summary and analysis of the preamble and questions that will be considered, and potential impact to larger financial institutions regarding executive compensation arrangements. While the client alert should showcase my expertise and my firm's capabilities, it should be digestible, similar in tone to conversation at a business lunch, and not use jargon.
    '''
)
system_prompt1 = ChatMessage(
        role='system_prompt',
        content=''' 
Draft a client alert about the proposed rule and request for public comment in the document. Client alert is a three page write up that will be published online and sent to existing and potential clients to highlight my knowledge, insight, and expertise. 
The Proposed Rule revives a proposal first made in 2016 regarding rules on incentive-based compensation arrangements at certain financial institutions with at least $1 billion in assets.

IMPORTANT: 
  * The Proposed Rule contains a preamble under the heading of "Overview of the 2024 Proposed Rule" that proposes certain alternatives and questions that will be considered for the final rule. 
  * The information in this preamble under the heading of "Overview of the 2024 Proposed Rule" and the questions that will be considered for the final rule are very important to address in this client memo. 
  * The information in the preamble under the heading of "Overview of the 2024 Proposed Rule" should be discussed in the most detail. 
  * The draft should provide as much detail as possible, include and explain all facts and conclusions derived from the document, and present all supporting information and analysis
  * Use bullet points where appropriate. 
  * While the answer should showcase the expertise and firm's capabilities, it should be digestible, similar in tone to conversation at a business lunch, and not use jargon.
    '''
)
client_prompts1 = [
    ChatMessage(
        role='user',
        content='''
        What are the key provisions of the proposed rule? Provide as much detail and context as possible. Be extremely detail oriented, verbose, and include all supporting information and analysis.
 
        '''
    ),
    ChatMessage(
        role='user',
        content=''' 
        Write a summary and analysis of the preamble and questions that will be considered. Provide as much detail and context as possible. Be extremely detail oriented, verbose, and include all supporting information and analysis.
        '''
    ),
    ChatMessage(
        role='user',
        content='''
        What is the potential impact to larger financial institutions regarding executive compensation arrangements? Provide as much detail and context as possible. Be extremely detail oriented, verbose, and include all supporting information and analysis.
        '''
    ),
    ChatMessage(
        role='user',
        content=''' 
        Draft three page client alert, based on the last 3 answers, making sure to include the key provisions of the proposed rule, a summary and analysis of the preamble and questions that will be considered, and potential impact to larger financial institutions regarding executive compensation arrangements. Provide as much detail and context as possible. Be extremely detail oriented, verbose, and include all supporting information and analysis.
        '''
    ),
]

### PDF2 Use Case

In [52]:
initial_prompt2 = ChatMessage(
    role='user',
    content = '''
    What's a no-shop provision? How often are they included in private acquisition agreements? Does the agreement attached contain a no-shop and if so, what section is it?
    '''
)
system_prompt2 = ChatMessage(
    role='system',
    content = '''
    You are an expert in securities and asset management law. Please provide a detailed explanation of each question in the document.  
    The document is a SECURITIES AND ASSET PURCHASE AGREEMENT.
    
    Important directions when answering the questions:
     * The answers should contain as much detail as possible.
     * Include and explain all facts and conclusions derived from the document
     * Present all supporting information and analysis
     * Provide all references from the document including article number, section number, and title. 
    '''
)
client_prompts2 = [
    ChatMessage( 
        role='user',
        content = "What's a no-shop provision? "
    ),
    ChatMessage(
        role='user', 
        content="How often are they included in private acquisition agreements?"
    ),
    ChatMessage(
        role='user',
        content="Does the agreement attached contain a no-shop and if so, what section is it?"
    ),
    ChatMessage(
        role='user',
        content="If the provision is included, what are the key terms and conditions of the no-shop provision?"
    )
]

### PDF3 Use Case

In [53]:
initial_prompt3 = ChatMessage(
    role='user',
    content = '''
    Please provide a detailed and thorough tabular summary of the economic "waterfall" provision in this LPA, integrating all defined terms and cross-referenced provisions into a single, plain english description of the economics. The audience is a sophisticated business person with a doctorate in finance.
    '''
)
system_prompt3 = ChatMessage(
    role='system',
    content = '''
    The audience is a sophisticated business person with a doctorate in finance.
    The output should be tabular.
    '''
)
client_prompts3 = [
    ChatMessage(
        role='user',
        content = "Provide a detailed and thorough tabular summary of the economic 'waterfall' provision in this LPA, integrating all defined terms and cross-referenced provisions into plain english description of the economics. Add the metrics of the economic 'waterfall' provisions to the table.",
    )
]

### PDF4 Use Case

In [54]:
initial_prompt4 = ChatMessage(
    role='user',
    content = '''
    Draft a detailed email to the general counsel of AavantiBio summarizing the main interim operating covenants that restrict AavantiBio in the period between signing and closing.
    ''',
)
system_prompt4 = ChatMessage(
    role='system',
    content = '''
    Draft emails in formal business language following legal standards.
    ''',
)
client_prompts4 = [
    ChatMessage(
        role='user',
        content = "Draft a detailed email to the general counsel of AavantiBio summarizing the main interim operating covenants that restrict AavantiBio in the period between signing and closing.",
    )
]

## Setup Workflow

In [55]:
MODEL_LARGE = "jamba-1.5-large"
MODEL_MINI = "jamba-1.5-mini"

use_cases = {
    "PDF1": {'initial_prompt': initial_prompt1, 'system_prompt': system_prompt1, 'user_prompt': client_prompts1, 'docs': [pdf1_doc]},
    "PDF2": {'initial_prompt': initial_prompt2, 'system_prompt': system_prompt2, 'user_prompt': client_prompts2, 'docs': [pdf2_doc]},
    "PDF3": {'initial_prompt': initial_prompt3, 'system_prompt': system_prompt3, 'user_prompt': client_prompts3, 'docs': [pdf3_doc]},
    "PDF4": {'initial_prompt': initial_prompt4, 'system_prompt': system_prompt4, 'user_prompt': client_prompts4, 'docs': [pdf4_doc]},
}

# run prompts as chat questions and return single response
def run_use_case(use_case, model, client):
    documents = use_case['docs']

    messages = [
        use_case['system_prompt']
    ]

    for q in use_case['user_prompt']:
        messages.append(q)
        response = client.chat.completions.create(
            messages=messages, 
            model=model,
            max_tokens=4096,
            temperature=0.1,
            documents=documents
        )
        response_message = response.choices[0].message
        pprint(f"Q: {q.content}")
        pprint(f"A: {response_message.content}")
        messages.append(response_message)

    full_question = '\n'.join([r.content for r in messages if r.role == 'system' or r.role == 'user'])
    full_response = '\n'.join([r.content for r in messages if r.role == 'assistant'])

    return full_question, full_response

## Document Workflow

**IMPORTANT:**
From this point the workflow is the same for all documents. Instead of code duplication for each pdf (out of 4 totals) we use variable
```
use_case_name = "PDF1"
```
Simply change its value to `'PDF2'`, `'PDF3'`, or `'PDF4'` to run flow for respective use case (based on pdf).

There are 3 ways to execute each use case:
1. using initial prompt
2. using enhanced (engineered) prompts - in case when they are chat-like they get concatenated into single prompt
3. using stacking messages that simulates chat: this simulates interaction by the user asking initial prompt's questions in sequence

### Initial Prompt

Run intial prompt given to have baseline answer.

In [56]:
use_case_name = "PDF1"
use_case = use_cases[use_case_name]
response = client.chat.completions.create(
    messages=[use_case['initial_prompt']],
    # model=MODEL_LARGE,
    model=MODEL_MINI,
    max_tokens=4096,
    temperature=0.1,
    documents=use_case['docs']
)

print(response.choices[0].message.content)

**Client Alert: Key Provisions and Potential Impact of the Proposed Rule on Incentive-Based Compensation Arrangements**

Dear [Client Name],

We are pleased to share with you an important update regarding the proposed rule on incentive-based compensation arrangements at certain financial institutions. This rule, which has been revived from its initial proposal in 2016, seeks to implement regulations under section 956 of the Dodd-Frank Wall Street Reform and Consumer Protection Act. The proposed rule applies to financial institutions with at least $1 billion in assets and aims to address the potential risks associated with incentive-based compensation arrangements.

**Key Provisions of the Proposed Rule**

The proposed rule includes several key provisions designed to mitigate the risks associated with incentive-based compensation arrangements. These provisions include:

1. **Prohibition on Certain Compensation Arrangements**: The rule prohibits incentive-based compensation arrangements 

| **Stage** | **Recipient** | **Distribution** | **Details** |
| --- | --- | --- | --- |
| **1. Initial Distribution** | Limited Partners | 100% of NDC | Limited Partners receive 100% of NDC until they have received an amount equal to an annual rate of 9%, compounded annually, on their aggregate unreturned Capital Contributions allocable to the Investment. |
| **2. Subsequent Distribution** | Limited Partners | 100% of NDC | Limited Partners receive 100% of NDC pro rata until they have received an amount equal to their unreturned Capital Contributions allocated to the Investment plus their allocable share of all Management Fees and other expenses paid by the Partnership from Partnership cash flow rather than from Capital Contributions. |
| **3. General Partner Distribution** | General Partner | 50% of NDC | 50% of NDC is allocated to the General Partner until it has received a cumulative distribution equal to 20% of all distributions made in the previous stages. |
| **4. Final Distribution** | Limited Partners and General Partner | 80% to Limited Partners, 20% to General Partner | 80% of NDC is allocated to the Limited Partners pro rata based on their respective Percentage Interests, and 20% is allocated to the General Partner. |
| **5. Over-Distribution** | General Partner | Return of Over-Distribution | If the General Partner has received an Over-Distribution, it must return the excess amount to the Partnership, which will then be distributed to the applicable Limited Partner(s) in accordance with Section 14.1(b). |
| **6. Redemption of Interests** | General Partner and Affiliates | Redemption Amount | Upon removal of the General Partner, its Carried Interest and Capital Interest, and the Interests of its Affiliates, will be redeemed for an amount equal to their Fair Market Value immediately prior to removal, payable either in cash or by a promissory note. |
| **7. Bankruptcy of General Partner** | General Partner |

### Enhanced Prompts (Concatenated)

Because some use cases have prompts defined as a sequence of questions (multi-turn chat) we can simply concatentate them together (including system prompt) and run as a consolidated prompt.

In [57]:
use_case = use_cases["PDF4"]
# concatenate all prompts into a single message
all_prompts = [use_case['system_prompt']] + use_case['user_prompt']
docs = use_case['docs']
user_single_prompt = ChatMessage(
    role='user',
    content = '\n'.join([prompt.content for prompt in all_prompts])
)

response = client.chat.completions.create(
    messages = [user_single_prompt],
    model=MODEL_LARGE,
    max_tokens=4096,
    temperature=0.1,
    documents=docs
)
print(all_prompts)
print("==============================================")
print(response.choices[0].message.content)

[ChatMessage(role='system', content='\n    Draft emails in formal business language following legal standards.\n    '), ChatMessage(role='user', content='Draft a detailed email to the general counsel of AavantiBio summarizing the main interim operating covenants that restrict AavantiBio in the period between signing and closing.')]
Subject: Summary of Interim Operating Covenants for AavantiBio

Dear [General Counsel's Name],

I hope this message finds you well. I am writing to provide a detailed summary of the main interim operating covenants that will restrict AavantiBio's operations during the period between signing and closing as outlined in the Agreement and Plan of Merger dated September 29, 2022.

1. **Operation of Business**:


  * AavantiBio must conduct its operations only in the ordinary course of business and in compliance with all applicable laws.
  * The company must use commercially reasonable efforts to preserve its current business organization, physical assets, and rel

### Use Cases via Chat with Stacked Messages (where applicable)

Use function `run_use_case` to call models with each message accumulating responses (stacking) like chat messages.
In the end all responses are concatenated into single response.

In [58]:
use_case_name = "PDF4"
use_case = use_cases[use_case_name]
    
full_question, full_response = run_use_case(use_case, MODEL_LARGE, client)

use_cases[use_case_name].update({'full_question': full_question, 'full_response': full_response})

('Q: Draft a detailed email to the general counsel of AavantiBio summarizing '
 'the main interim operating covenants that restrict AavantiBio in the period '
 'between signing and closing.')
('A: Subject: Summary of Interim Operating Covenants for AavantiBio\n'
 '\n'
 "Dear [General Counsel's Name],\n"
 '\n'
 'I hope this email finds you well. I am writing to provide a detailed summary '
 "of the main interim operating covenants that will restrict AavantiBio's "
 'operations during the period between signing and closing, as outlined in the '
 'Agreement and Plan of Merger dated September 29, 2022.\n'
 '\n'
 "1. **Operation of AavantiBio's Business**:\n"
 '\n'
 '\n'
 '  * AavantiBio must use commercially reasonable efforts to conduct its '
 'operations in the ordinary course of business and in compliance with all '
 'applicable laws.\n'
 '  * The company must preserve its current business organization, maintain '
 'its physical assets, and keep available the services of its current off

In [59]:
print(use_cases[use_case_name]['full_question'])
print('====================================')
print(use_cases[use_case_name]['full_response'])
print('====================================')


    Draft emails in formal business language following legal standards.
    
Draft a detailed email to the general counsel of AavantiBio summarizing the main interim operating covenants that restrict AavantiBio in the period between signing and closing.
Subject: Summary of Interim Operating Covenants for AavantiBio

Dear [General Counsel's Name],

I hope this email finds you well. I am writing to provide a detailed summary of the main interim operating covenants that will restrict AavantiBio's operations during the period between signing and closing, as outlined in the Agreement and Plan of Merger dated September 29, 2022.

1. **Operation of AavantiBio's Business**:


  * AavantiBio must use commercially reasonable efforts to conduct its operations in the ordinary course of business and in compliance with all applicable laws.
  * The company must preserve its current business organization, maintain its physical assets, and keep available the services of its current officers and employ

| **Metric** | **Description** | **Defined Terms and Cross-Referenced Provisions** |
| --- | --- | --- |
| **Net Distributable Cash** | All cash receipts from operations and capital events, reduced by principal or interest on any indebtedness, reserves, and operating expenses. | Net Distributable Cash, Operating Expenses, Capital Events |
| **Distribution of Net Distributable Cash** | Calculated on an investment-by-investment basis and allocated among partners in proportion to their respective percentage interests. | Percentage Interest, Capital Events |
| **Distribution to General Partner** | The full amount of Net Distributable Cash allocated to the General Partner or its affiliates is distributed to them. | General Partner, Affiliates |
| **Distribution to Limited Partners** | Allocated in the following manner: <br> 1. 100% to the Limited Partner until it has received an amount equal to an annual rate of 9% on its aggregate unreturned capital contributions allocable to the investment. <br> 2. 100% to the Limited Partner pro rata until it has received an amount equal to its unreturned capital contributions plus its allocable share of management fees and other expenses. <br> 3. 50% to the Limited Partner and 50% to the General Partner until the General Partner has received 20% of all distributions. <br> 4. 80% to the Limited Partner and 20% to the General Partner. | Limited Partner, Capital Contributions, Management Fees, General Partner |
| **Returned Capital** | Amount of Net Distributable Cash which represents the return of the Partnership’s invested capital in any investment that has been disposed of or refinanced. | Returned Capital, Capital Contributions |
| **Management Fee** | Paid to the General Partner quarterly in arrears, based on each Limited Partner’s capital commitment during the investment period and each Limited Partner’s invested capital after the investment period. | Management Fee, Capital Commitment, Invested Capital |
| **Over-Distribution** | If the General Partner has received an Over-Distribution, it shall pay the amount to the Partnership, which shall then be distributed to the applicable Limited Partner(s). | Over-Distribution, Carried Interest Distributions |
| **Final Accounting and Clawback** | Within 120 days after the disposition of the last investment, the General Partner shall determine if there has been any Over-Distribution and shall pay the amount to the Partnership. | Final Accounting, Clawback, Over-Distribution |
| **Cancellation of Certificate** | Upon dissolution of the Partnership and completion of winding up, the General Partner shall cause the cancellation of the Certificate and terminate the Partnership. | Cancellation of Certificate, Dissolution |
| **Service of Process** | Each Partner consents to the service of process by mailing copies thereof, by certified mail, to such party at its address as set forth in Section 15.1. | Service of Process, Section 15.1 |
| **Trial** | Each of the Partnership and each Partner waives their rights to a jury trial of any claim or cause of action based upon or arising out of this Agreement. | Trial, Jury Trial |
